## ROS concepts explained Topic, Service, Parameter

Topic, Service, and Parameter are the main parts of ROS. Each simplifies some particular type of communication inside ROS so that your robots can do stuff efficiently.

In short:
**Topic** is used when we don't care who writes the message and who reads the message as long as it is on the right topic and of the right format (i.e. "What is the **temperature** outside?" - someone asks. "It is 5 **degrees Celcius**" answers someone) 

**Service** is a different format, where we ask for some calculation or action and receive response **in return**.
I.e. (I ask "how do you do, robot?", and I receive "fine, thank you").
Both command and response need to be in a format that we agreed upon. Also, there can be only one agent (i.e. one robot) with a particular service. 

**Parameter** is a common database of parameters that can be read or written by ROS nodes (programms). Such parameters may be the hardware configuration, robot description or some other settings. Usually, they are read by a node when initializing (starting).

In [3]:
### we will again connect to the main server

%env ROS_MASTER_URI=http://igor-laptop-linux:11311
import rospy

env: ROS_MASTER_URI=http://igor-laptop-linux:11311


### ROS Topic

Let's start from *ROS Topic*. You have used this protocol to read secret Ono message. **rostopic** program was smart enough to read this message automatically but normally we need to know what **message type** the topic has to read it. This is because the messages can be very different and actually have a whole structure inside. 
The message is **serialized** (coded) before being sent by **publisher** and then **deserialized** (decoded) by **subscriber**. The whole process is done automatically but we need to know the message type.

### Exercise: 
*Let's look again at the topics that are there. In the field below get the list of the topics with their message types in format [[topic_name,message_type],[... ]].*

In [2]:
topics = rospy.get_published_topics() # read the topics here
print(topics)

gaierror: [Errno -5] No address associated with hostname

*HINT:*
    You can find the function for finding published *topics* in the previous chapter: [setting_up_and_running_your_first_ROS_Node.ipynb](setting_up_and_running_your_first_ROS_Node.ipynb) 

You can see that there is topic called '/Ono/joint_state'. Let's read its  message stream using **rostopic** tool.

### Exercise:

*Read **Ono/joint_state** topic, using rostopic echo command, interupt kernel when fininshed*


In [3]:
!rostopic echo Ono/joint_state

header: 
  seq: 1
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
name: ['eye', 'eye_lid', 'hand', 'right_leg', 'left_leg']
position: [2.5232882751936936, 8.203797781022965, 2.8160240350570707, 0.9917657422693971, 2.2916228094909985]
velocity: []
effort: []
---
header: 
  seq: 2
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
name: ['eye', 'eye_lid', 'hand', 'right_leg', 'left_leg']
position: [7.626514390093025, 7.7912285932074745, 3.96186837417146, 6.303211402643686, 6.515375269864881]
velocity: []
effort: []
---
header: 
  seq: 3
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
name: ['eye', 'eye_lid', 'hand', 'right_leg', 'left_leg']
position: [0.5592027248691445, 9.750457531620984, 8.996208664380724, 2.784024007675028, 0.3917081192897298]
velocity: []
effort: []
---
header: 
  seq: 4
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
name: ['eye', 'eye_lid', 'hand', 'right_leg', 'left_leg']
position: [5.4276012110356175, 8.701349860478288, 4.2334

So this topic sends much more complicated messages. It uses one of common message types, called *Ono/joint_state*. As robots have multiple parts, connected using joints it is usually good to know how these joints are set. This can be used, for example, to visualize robot. The message has also a **header** that is a form of a **[timestamp](http://wiki.ros.org/ecl_time/Tutorials/Timestamps%20and%20Durations)** that gives us information when the message was sent. This is useful too, for example, use only the newest messages or even debug the communication if the header is strange or very old.

### Let's write our own publisher

Ok, let's publish some messages.

In [4]:
import rospy
from std_msgs.msg import String

#rospy... # initialize your node here! otherwise ROS will not know who and where your node is!
rospy.init_node("my_name", anonymous=True)

Let's initialize the Publisher -- an object that publishes to the topic "chatter" a message in format of String. 

In [18]:
my_publisher=rospy.Publisher("chatter",String,queue_size=10)

Ok, now let's publish some info. For simple messages like String, we can send the message super- easly.

In [30]:
my_publisher.publish("Kocham Misia!!!")

data: Kocham Misia!!!

[ERROR] [1502128045.486342]: bad callback: <function how_long_in_my_message at 0x7ff2da8eb950>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-15-65fa30820679>", line 3, in how_long_in_my_message
    len(message)
TypeError: object of type 'String' has no len()




data: Kocham Misia!!!
data: Kocham Misia!!!
Kocham Misia!!!


Ok but how to read what we have sent? One way is to use *rostopic echo* command but you cannot do this from Ipython this time because it will only show you the messages sent after you have started the program. You can do this by opening a Ubuntu terminal, using the icon on the left ![Ubuntu terminal icon](images/terminal.png)
or shortcut Ctrl+Alt+T. Than put the command 

```rostopic echo /chatter``` 


When you re-run the publish command in Ipython, you should see your message appearing in the terminal.

Ok, with rospublish command our node became a ros publisher.
We can see that using rosnode.get_node_info. To see the whole graph structure right now we can use graphical tool called rqt_graph (you can also start it in a seperate terminal)

In [9]:
!rqt_graph

What you see after running this command is whole ROS network graph. You can see your own node, among others, publishing on the /chatter topic. This type of communication allows for multiple senders and receivers.

### Let's write a Subscriber

**Subscriber** is an object that receives messages written on a particular topic. We want to receive all **calls** to our subscriber so we will register a **callback function** to do stuff when we receive a new message. The easiest thing we can do with an incoming message is to print it, so let's put a *print* function as our callback function.

After running the code below, you can check that your subscriber is working by using your previously defined publisher. Try to publish something to a /chatter topic!


In [11]:
my_subscriber = rospy.Subscriber("chatter",String,print)

data: I love Lidka
data: I love Lidka!
data: I love Lidka!


As soon as you create a new subscriber, you will start receiving a stream of messages and you will start to see the incoming messages. In case of "/chatter" topic they are published by your and other publishers, but imagine that there is an automatic publisher publishing every couple of messages. This could become overwhelming!

If you want to stop running the callback, use the unregister method.


In [10]:
my_subscriber.unregister()

[WARN] [1502126080.699939]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


### Exercise:
*Before we tackle this problem, try to write your own callback function to do something with received message. For example, read /how_long_is_my_message topic and print the length of the string*

*HINT 1:* You can define a function using the normal

    def some_function_name(message):
        the stuff you do

   and than pass the function _name_ as a callback so rospy.Subscriber(..., some_function_name)

*HINT 2:* Remember that the message is an object with a field data (message.data) that actually has the string itself


In [25]:
def how_long_in_my_message(message):
    #here do your magic change the print below to
    print(message.data)
    #len(message)
    pass

subscriber2= rospy.Subscriber("chatter",String,how_long_in_my_message) # here create a subscriber that uses your function

In [22]:
subscriber2.unregister()

In [26]:
# try your function using the publisher

my_publisher.publish("some test message")

data: some test message
data: some test message


[ERROR] [1502127291.162258]: bad callback: <function how_long_in_my_message at 0x7ff2da8eb950>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-15-65fa30820679>", line 3, in how_long_in_my_message
    len(message)
TypeError: object of type 'String' has no len()



data: some test message
some test message
data: I love Lidka!
data: I love Lidka!
data: I love Lidka!
I love Lidka!


[ERROR] [1502127543.834674]: bad callback: <function how_long_in_my_message at 0x7ff2da8eb950>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-15-65fa30820679>", line 3, in how_long_in_my_message
    len(message)
TypeError: object of type 'String' has no len()



data: I love Lidka!
data: I love Lidka!
data: I love Lidka!
I love Lidka!


...

[ERROR] [1502127742.080575]: bad callback: <function how_long_in_my_message at 0x7ff2da8eb950>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-15-65fa30820679>", line 3, in how_long_in_my_message
    len(message)
TypeError: object of type 'String' has no len()



data: ❤❤❤❤❤ I love Lidka ❤❤❤❤❤!
data: ❤❤❤❤❤ I love Lidka ❤❤❤❤❤!
data: ❤❤❤❤❤ I love Lidka ❤❤❤❤❤!
❤❤❤❤❤ I love Lidka ❤❤❤❤❤!


[ERROR] [1502127748.429330]: bad callback: <function how_long_in_my_message at 0x7ff2da8eb950>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-15-65fa30820679>", line 3, in how_long_in_my_message
    len(message)
TypeError: object of type 'String' has no len()



data: ❤ ❤ ❤ ❤ ❤ I love Lidka  ❤ ❤ ❤ ❤ ❤!
data: ❤ ❤ ❤ ❤ ❤ I love Lidka  ❤ ❤ ❤ ❤ ❤!
data: ❤ ❤ ❤ ❤ ❤ I love Lidka  ❤ ❤ ❤ ❤ ❤!
❤ ❤ ❤ ❤ ❤ I love Lidka  ❤ ❤ ❤ ❤ ❤!


[ERROR] [1502127764.607935]: bad callback: <function how_long_in_my_message at 0x7ff2da8eb950>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-15-65fa30820679>", line 3, in how_long_in_my_message
    len(message)
TypeError: object of type 'String' has no len()



## Exercise: 

Ok, nice! 

*Now create a new publisher, to some choosen topic (you decide) and  use it to publish the length of received string. That is your new function will modify the incomming messages, something like:*

/how_long_is_my_message -> | your node | -> /message_length

*HINT:*
Call the newly created publisher from inside the callback function. 




In [ ]:
publisher= ... #here create a publisher to the new topic

def receive_and_modify(message): #here 
    pass


new_subscriber=...

Super nice! What you did is a common strategy used in ROS. Instead of one program that does all the computations, each node takes some messages from different topics and publishes your own. You connect them together and have a computation structure to, for example do robot localisation. 



### ROS  Service

Services are a common way to ask for some data/ calculation or change a state of the robot. 

Similar to ROS topic, both actors need to know the format of message. There will be actually two formats, format of question and format of the answer. 

Let's see how such format looks

In [14]:
!rossrv show std_srvs/SetBool

bool data
---
bool success
string message



What you can see is that for a question sent as a boolean data (True/False) 
the service will respond with a boolean response as well as a message for us.

Ono has a big diode, we will try to switch it on. To do this, we will use a **proxy function**, that is we will define something that uses the service but from our perspective looks like a normal function that _returns_ some value. Try running this function couple of times. 

As we can see, sometimes we suceed in setting up the diode, sometimes we fail, and ono explains why. This is a common scenario in robotics that things can go wrong and we have a choice of re running something or trying something else.

To write our own service, we use




### ROS Param

The last (common) way of communication between nodes is ROS Param. 

All nodes have access to a common _dictionary_ like structure, from which they can read and write parameters.

The typical scenario is that we write some setup parameters to ROS Parameter server instead than just to start a program with command line arguments. Other is to put elements that are used by multiple programs such as URDF robot models

Think of ROS Param as a place to put settings and common data of your ROS network. 

Using ROS Parameter is done via rosparam module

In [19]:
import rosparam

rosparam.list_params("/")

['/rosdistro',
 '/roslaunch/uris/host_igor_laptop_linux__57314',
 '/rosversion',
 '/run_id']